In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from source.utils.load_utils import just_load
from source.datasets.pretrain_datasets import MSDDatasetPretrain
from source.models_pretraining.mb_mlm import MusicBertMaskedLanguageModeling

# REVIEWS DATASET
song_dataset = MSDDatasetPretrain()

train_dataloader = just_load(song_dataset, workers=8, batch_size=4)

evals = 1
model = MusicBertMaskedLanguageModeling(CLS_training='SM', 
                                        MLM_training='CE',
                                        num_encoder_layers=4
                                       ).cuda()
#model.load_pretrained()

In [ ]:
model.pretrain_model(train_dataloader, None, epochs=1, eval_per_epoch=evals)

In [2]:
%matplotlib widget

from source.utils.plot_utils import plot_curve

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

#cls_loss = smooth(model.cls_loss_curve.cpu().numpy(), 100)[100:-100]

plot_curve(model.mask_loss_curve.cpu(), 1, color="red")
#plot_curve(cls_loss, 1, color="blue")

NameError: name 'model' is not defined

In [3]:
%load_ext line_profiler

%lprun -f MSDDatasetPretrain.__getitem__ model.pretrain_model(train_dataloader, None, epochs=1, eval_per_epoch=evals)

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-06 s

Total time: 33.0413 s
File: /nfs/notebooks/source/dataset_pretrain.py
Function: __getitem__ at line 42

Line #      Hits         Time  Per Hit   % Time  Line Contents
    42                                               def __getitem__(self, idx):
    43                                                   
    44      1843       4761.0      2.6      0.0          msd_id = self.msd_ids_list[idx][0:17] #Cutoff to comply with moodagent modified IDs
    45      1843       1842.0      1.0      0.0          msd_id = msd_id.encode()
    46                                                   
    47      1843       7916.0      4.3      0.0          with self.song_env.begin(write=False) as txn:
    48      1843   32982999.0  17896.4     99.8              data = txn.get(msd_id)
    49                                                       
    50      1842       2771.0      1.5      0.0          assert data is not None, "Key Error in database"
    51                               